In [13]:
import os
from pathlib import Path

import pandas as pd
from typing import Tuple, Union

import torch
import torchaudio

from torch.utils.data import Dataset
from torchaudio._internal import download_url_to_file
from torchaudio.datasets.utils import _extract_zip

ImportError: cannot import name 'download_url_to_file' from 'torchaudio._internal' (/opt/conda/lib/python3.9/site-packages/torchaudio/_internal/__init__.py)

In [10]:
_RELEASE_CONFIGS = {
    'release1': {
        "folder_in_archive": "the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data",
        "url": "https://physionet.org/static/published-projects/circor-heart-sound/the-circor-digiscope-phonocardiogram-dataset-1.0.3.zip"
    }
}

class Phonocardiogram(Dataset):
    def __init__(
        self,
        root: Union[str, Path],
        url: str = _RELEASE_CONFIGS['release1']['url'],
        folder_in_archive: str = _RELEASE_CONFIGS['release1']['folder_in_archive'],
        download: bool = False,
    ) -> None:
        self._root = Path(root)
        self._dataset_dir = self._root / folder_in_archive
        
        archive = os.path.basename(url)
        archive = self._root / archive
        
        if download:
            if not os.path.isdir(self._dataset_dir):
                if not os.path.isfile(archive):
                    download_url_to_file(url, archive)I 
                _extract_zip(archive)
        
        if not os.path.isdir(self._dataset_dir):
            raise RuntimeError("Dataset not found. Please use `download=True` to download it.")
        
        self._patient_ids = [p.stem for p in self._dataset_dir.glob('*.wav')]
    
    def __getitem__(self, n: int) -> Tuple[torch.Tensor, int, torch.Tensor, torch.Tensor]:
        patient_id = self._patient_ids[n]
        wav_path = self._dataset_dir / (patient_id + '.wav')
        tsv_path = self._dataset_dir / (patient_id + '.tsv')
        
        waveform, sample_rate = torchaudio.load(wav_path)
        
        labels_df = pd.read_csv(tsv_path, delimiter='\t', header=None)
        s1_ranges = torch.tensor(labels_df[labels_df[2] == 1].drop(2, axis=1).values)
        s2_ranges = torch.tensor(labels_df[labels_df[2] == 3].drop(2, axis=1).values)
        
        return waveform, sample_rate, s1_ranges, s2_ranges
    
    def __len__(self) -> int:
        return len(self._patient_ids)

In [12]:
dataset = Phonocardiogram(root='dataset', download=True)

len(dataset)

NameError: name 'download_url_to_file' is not defined

In [ ]:
waveform, sample_rate, s1, s2 = dataset[1]

In [ ]:
print(waveform.shape, s1.shape, s2.shape)

In [ ]:
waveform, sample_rate, s1, s2